# ZarrManager example

In [1]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time

In [2]:
# --- Set input data path ---
main_path = "../../pymif_test_data/zarr"
dataset_name = "5T_2C_81Z_2304Y_2304X"
dataset_name = "viventis_5T_2C_8Z_128Y_128X"
input_zarr_path = f"{main_path}/{dataset_name}.zarr"


In [3]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: None = zarr native chunks)
zarr_dataset = mm.ZarrManager(path=input_zarr_path)

In [4]:
# --- Show metadata summary ---
for i in zarr_dataset.metadata:
    print(f"{i.upper()}: {zarr_dataset.metadata[i]}")

SIZE: [(5, 2, 8, 128, 128), (5, 2, 4, 64, 64), (5, 2, 2, 32, 32), (5, 2, 1, 16, 16), (5, 2, 1, 8, 8)]
CHUNKSIZE: [(1, 1, 8, 128, 128), (1, 1, 4, 64, 64), (1, 1, 2, 32, 32), (1, 1, 1, 16, 16), (1, 1, 1, 8, 8)]
SCALES: [(4.0, 0.692, 0.692), (8.0, 1.384, 1.384), (16.0, 2.768, 2.768), (32.0, 5.536, 5.536), (64.0, 11.072, 11.072)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 900.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['Hoechst', 'FM4-64']
CHANNEL_COLORS: [16711935, 255]
DTYPE: uint16
PLANE_FILES: None
AXES: tczyx


In [5]:
zarr_dataset.data

[dask.array<from-zarr, shape=(5, 2, 8, 128, 128), dtype=uint16, chunksize=(1, 1, 8, 128, 128), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(5, 2, 4, 64, 64), dtype=uint16, chunksize=(1, 1, 4, 64, 64), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(5, 2, 2, 32, 32), dtype=uint16, chunksize=(1, 1, 2, 32, 32), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(5, 2, 1, 16, 16), dtype=uint16, chunksize=(1, 1, 1, 16, 16), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(5, 2, 1, 8, 8), dtype=uint16, chunksize=(1, 1, 1, 8, 8), chunktype=numpy.ndarray>]

## Adding labels

In [6]:
import dask.array as da
from skimage.measure import label
import numpy as np

# Function that runs labeling on numpy blocks
def label_block(block):
    return label(block > 1000)

# Use map_blocks for each channel
label_images = []
for channel in [0, 1]:

    label_levels = []
    
    for t in range(zarr_dataset.data[0].shape[0]):
        
        slice_t = zarr_dataset.data[0][t, channel]
        
        labeled = da.map_overlap(
            label_block,
            slice_t,
            depth={0: 1, 1: 10, 2: 10},  # no overlap in Z, 10 px in Y and X
            boundary='reflect',
            trim=True,
            dtype=np.int32,
        )
        
        # Add back T-C axis
        labeled = labeled[None, None, ...]
        label_levels.append(labeled)
    
    # Stack back to full (t, z, y, x)
    label_stack = da.concatenate(label_levels, axis=0)

    label_images.append([label_stack])

label_names = ["nuclei", "membranes"]

In [7]:
# Create pyramid compliant with raw data dimensions
from pymif.microscope_manager.utils import pyramid
label_images = [
    pyramid.build_pyramid(label_images[0],
                         zarr_dataset.metadata,
                         num_levels = 5)[0],
    pyramid.build_pyramid(label_images[1],
                         zarr_dataset.metadata,
                         num_levels = 5)[0]
]

# Remove channel dimension from labels
label_images = [
    [l[:,0] for l in label_images[0]],
    [l[:,0] for l in label_images[1]],
]

Requested start level 0
Resolution layer already available.
Creating pyramid.
Updating metadata.
Requested start level 0
Resolution layer already available.
Creating pyramid.
Updating metadata.


In [8]:
# Add labels to the dataset
zarr_dataset.add_label(label_images[0], label_names[0])
zarr_dataset.add_label(label_images[1], label_names[1])

c:\miniforge3\envs\pymif\lib\site-packages\pymif\microscope_manager\utils\add_label.py:75: FutureWarning: The NestedDirectoryStore is deprecated and will be removed in a Zarr-Python version 3, see https://github.com/zarr-developers/zarr-python/issues/1274 for more information.
  store = zarr.NestedDirectoryStore(str(store_path))


In [9]:
# Reload dataset
zarr_dataset = mm.ZarrManager(path=input_zarr_path, chunks=(1,1,64,1024,1024))

In [10]:
zarr_dataset.data

[dask.array<from-zarr, shape=(5, 2, 8, 128, 128), dtype=uint16, chunksize=(1, 1, 8, 128, 128), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(5, 2, 4, 64, 64), dtype=uint16, chunksize=(1, 1, 4, 64, 64), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(5, 2, 2, 32, 32), dtype=uint16, chunksize=(1, 1, 2, 32, 32), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(5, 2, 1, 16, 16), dtype=uint16, chunksize=(1, 1, 1, 16, 16), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(5, 2, 1, 8, 8), dtype=uint16, chunksize=(1, 1, 1, 8, 8), chunktype=numpy.ndarray>]

In [11]:
zarr_dataset.labels

{'membranes': [dask.array<from-zarr, shape=(5, 8, 128, 128), dtype=int32, chunksize=(1, 8, 128, 128), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(5, 4, 64, 64), dtype=int32, chunksize=(1, 4, 64, 64), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(5, 2, 32, 32), dtype=int32, chunksize=(1, 2, 32, 32), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(5, 1, 16, 16), dtype=int32, chunksize=(1, 1, 16, 16), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(5, 1, 8, 8), dtype=int32, chunksize=(1, 1, 8, 8), chunktype=numpy.ndarray>],
 'nuclei': [dask.array<from-zarr, shape=(5, 8, 128, 128), dtype=int32, chunksize=(1, 8, 128, 128), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(5, 4, 64, 64), dtype=int32, chunksize=(1, 4, 64, 64), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(5, 2, 32, 32), dtype=int32, chunksize=(1, 2, 32, 32), chunktype=numpy.ndarray>,
  dask.array<from-zarr, shape=(5, 1, 16, 16), dtype=int32, chunksize=(1, 1, 16, 16),

In [12]:
zarr_dataset.metadata

{'size': [(5, 2, 8, 128, 128),
  (5, 2, 4, 64, 64),
  (5, 2, 2, 32, 32),
  (5, 2, 1, 16, 16),
  (5, 2, 1, 8, 8)],
 'chunksize': [(1, 1, 8, 128, 128),
  (1, 1, 4, 64, 64),
  (1, 1, 2, 32, 32),
  (1, 1, 1, 16, 16),
  (1, 1, 1, 8, 8)],
 'scales': [(4.0, 0.692, 0.692),
  (8.0, 1.384, 1.384),
  (16.0, 2.768, 2.768),
  (32.0, 5.536, 5.536),
  (64.0, 11.072, 11.072)],
 'units': ('micrometer', 'micrometer', 'micrometer'),
 'time_increment': 900.0,
 'time_increment_unit': 's',
 'channel_names': ['Hoechst', 'FM4-64'],
 'channel_colors': [16711935, 255],
 'dtype': 'uint16',
 'plane_files': None,
 'axes': 'tczyx',
 'labels_metadata': {'membranes': {'data': [dask.array<from-zarr, shape=(5, 8, 128, 128), dtype=int32, chunksize=(1, 8, 128, 128), chunktype=numpy.ndarray>,
    dask.array<from-zarr, shape=(5, 4, 64, 64), dtype=int32, chunksize=(1, 4, 64, 64), chunktype=numpy.ndarray>,
    dask.array<from-zarr, shape=(5, 2, 32, 32), dtype=int32, chunksize=(1, 2, 32, 32), chunktype=numpy.ndarray>,
    das

In [13]:
v = zarr_dataset.visualize(stop_level=2, in_memory=True)

In [14]:
zarr_dataset.visualize_zarr()

Viewer(camera=Camera(center=(0.0, 43.942, 43.942), zoom=6.435151734104046, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True, orientation=(<DepthAxisOrientation.TOWARDS: 'towards'>, <VerticalAxisOrientation.DOWN: 'down'>, <HorizontalAxisOrientation.RIGHT: 'right'>)), cursor=Cursor(position=(1800.0, 12.0, 0.0, 0.0), viewbox=None, scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=6.92), dims=Dims(ndim=4, ndisplay=2, order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3'), rollable=(True, True, True, True), range=(RangeTuple(start=0.0, stop=3600.0, step=900.0), RangeTuple(start=0.0, stop=28.0, step=4.0), RangeTuple(start=0.0, stop=87.884, step=0.692), RangeTuple(start=0.0, stop=87.884, step=0.692)), margin_left=(0.0, 0.0, 0.0, 0.0), margin_right=(0.0, 0.0, 0.0, 0.0), point=(1800.0, 12.0, 43.596, 43.596), last_used=0), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False, spacing=0.0), layers=[<Image layer 'Hoechst' at 0x21d818db580>, <Image layer 'FM4-64'